Google data downloaded from https://transparencyreport.google.com/political-ads/region/US.
Specific file used is google-political-ads-creative-stats.csv

Zip Code data downloaded from http://federalgovernmentzipcodes.us/download.html

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
data = pd.read_csv('google-political-ads-creative-stats.csv')
data.columns

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Index(['Ad_ID', 'Ad_URL', 'Ad_Type', 'Regions', 'Advertiser_ID',
       'Advertiser_Name', 'Ad_Campaigns_List', 'Date_Range_Start',
       'Date_Range_End', 'Num_of_Days', 'Impressions', 'Spend_USD',
       'First_Served_Timestamp', 'Last_Served_Timestamp', 'Age_Targeting',
       'Gender_Targeting', 'Geo_Targeting_Included', 'Geo_Targeting_Excluded',
       'Spend_Range_Min_USD', 'Spend_Range_Max_USD', 'Spend_Range_Min_EUR',
       'Spend_Range_Max_EUR', 'Spend_Range_Min_INR', 'Spend_Range_Max_INR',
       'Spend_Range_Min_BGN', 'Spend_Range_Max_BGN', 'Spend_Range_Min_HRK',
       'Spend_Range_Max_HRK', 'Spend_Range_Min_CZK', 'Spend_Range_Max_CZK',
       'Spend_Range_Min_DKK', 'Spend_Range_Max_DKK', 'Spend_Range_Min_HUF',
       'Spend_Range_Max_HUF', 'Spend_Range_Min_PLN', 'Spend_Range_Max_PLN',
       'Spend_Range_Min_RON', 'Spend_Range_Max_RON', 'Spend_Range_Min_SEK',
       'Spend_Range_Max_SEK', 'Spend_Range_Min_GBP', 'Spend_Range_Max_GBP',
       'Spend_Range_Min_ILS', 'Spend_R

In [3]:
data.head()

,Ad_ID,Ad_URL,Ad_Type,Regions,Advertiser_ID,Advertiser_Name,Ad_Campaigns_List,Date_Range_Start,Date_Range_End,Num_of_Days,...,Spend_Range_Min_GBP,Spend_Range_Max_GBP,Spend_Range_Min_ILS,Spend_Range_Max_ILS,Spend_Range_Min_NZD,Spend_Range_Max_NZD,Spend_Range_Min_TWD,Spend_Range_Max_TWD,Spend_Range_Min_AUD,Spend_Range_Max_AUD
0,CR104193501419274240,https://transparencyreport.google.com/politica...,Image,"EU, IT",AR100125170957549568,Azione,NaN,2020-10-26,2020-11-25,31,...,0,50.0,0,250.0,0,200.0,0,3000.0,0,150.0
1,CR109957691127889920,https://transparencyreport.google.com/politica...,Video,"EU, IT",AR100125170957549568,Azione,NaN,2020-12-02,2020-12-03,2,...,0,50.0,0,250.0,0,200.0,0,3000.0,0,150.0
2,CR111129083328331776,https://transparencyreport.google.com/politica...,Image,"EU, IT",AR100125170957549568,Azione,NaN,2020-06-28,2020-09-03,35,...,500,25000.0,2500,125000.0,1500,75000.0,30000,1500000.0,1500,50000.0
3,CR112145169511350272,https://transparencyreport.google.com/politica...,Image,"EU, IT",AR100125170957549568,Azione,NaN,2020-10-26,2020-11-02,8,...,0,50.0,0,250.0,0,200.0,0,3000.0,0,150.0
4,CR112255773509156864,https://transparencyreport.google.com/politica...,Image,"EU, IT",AR100125170957549568,Azione,NaN,2021-01-28,2021-02-04,8,...,50,500.0,250,2500.0,200,1500.0,3000,30000.0,150,1500.0


In [4]:
data.dtypes

Ad_ID                      object
Ad_URL                     object
Ad_Type                    object
Regions                    object
Advertiser_ID              object
Advertiser_Name            object
Ad_Campaigns_List         float64
Date_Range_Start           object
Date_Range_End             object
Num_of_Days                 int64
Impressions                object
Spend_USD                  object
First_Served_Timestamp     object
Last_Served_Timestamp      object
Age_Targeting              object
Gender_Targeting           object
Geo_Targeting_Included     object
Geo_Targeting_Excluded     object
Spend_Range_Min_USD         int64
Spend_Range_Max_USD       float64
Spend_Range_Min_EUR         int64
Spend_Range_Max_EUR       float64
Spend_Range_Min_INR         int64
Spend_Range_Max_INR       float64
Spend_Range_Min_BGN         int64
Spend_Range_Max_BGN       float64
Spend_Range_Min_HRK         int64
Spend_Range_Max_HRK       float64
Spend_Range_Min_CZK         int64
Spend_Range_Ma

In [5]:
data['Ad_Campaigns_List'].unique()

array([nan])

In [6]:
#Remove non US spending ranges, Ad_Id, Ad_URL, Advertiser_ID (not useful to us), specific timestamps 
#(already has number of days with dates), and Ad_Campaigns_List
columns_of_interest = ['Ad_Type', 'Regions', 'Advertiser_Name', 'Date_Range_Start',
       'Date_Range_End', 'Num_of_Days', 'Impressions', 'Spend_USD','Age_Targeting',
       'Gender_Targeting', 'Geo_Targeting_Included', 'Geo_Targeting_Excluded',
       'Spend_Range_Min_USD', 'Spend_Range_Max_USD']

#Select US ads
data = data[columns_of_interest]
data = data.loc[data['Regions'] == 'US']

In [7]:
#Select ads that cost at least $1,000
print(data['Spend_Range_Min_USD'].value_counts())
data = data.loc[data['Spend_Range_Min_USD'] >= 1000]

0         392518
100       131844
1000       79110
50000       1261
100000       857
Name: Spend_Range_Min_USD, dtype: int64


In [8]:
data.head()

,Ad_Type,Regions,Advertiser_Name,Date_Range_Start,Date_Range_End,Num_of_Days,Impressions,Spend_USD,Age_Targeting,Gender_Targeting,Geo_Targeting_Included,Geo_Targeting_Excluded,Spend_Range_Min_USD,Spend_Range_Max_USD
293,Text,US,ALEX FOR AZ,2020-07-08,2020-08-04,28,≤ 10k,1k-50k,Not targeted,Not targeted,"85054, 85207, 85215, 85250, 85251, 85253, 8525...",Not targeted,1000,50000.0
296,Text,US,ALEX FOR AZ,2020-07-07,2020-08-04,29,≤ 10k,1k-50k,Not targeted,Not targeted,"85054, 85207, 85215, 85250, 85251, 85253, 8525...",Not targeted,1000,50000.0
303,Text,US,CITIZENS FOR JD WOOTEN,2020-08-28,2020-11-03,64,10k-100k,1k-50k,Not targeted,Not targeted,"27215, 27217, 27233, 27244, 27249, 27253, 2725...",Not targeted,1000,50000.0
304,Video,US,CITIZENS FOR JD WOOTEN,2020-10-28,2020-11-03,7,100k-1M,1k-50k,"18-24, 25-34, 35-44, 45-54, 55-64, ≥65, Unknow...","Male, Female, Unknown gender","27215, 27217, 27233, 27244, 27249, 27253, 2725...",Not targeted,1000,50000.0
305,Video,US,CITIZENS FOR JD WOOTEN,2020-10-26,2020-11-03,9,10k-100k,1k-50k,"18-24, 25-34, 35-44, 45-54, 55-64, ≥65, Unknow...","Male, Female, Unknown gender","27215, 27217, 27233, 27244, 27249, 27253, 2725...",Not targeted,1000,50000.0


In [11]:
#change date columns to datetime format
data.loc[:, 'Date_Range_Start'] = pd.to_datetime(data['Date_Range_Start'])
data.loc[:, 'Date_Range_End'] = pd.to_datetime(data['Date_Range_End'])

In [50]:
data.to_csv('export_dataframe.csv', index = False, header=True)

In [9]:
clean_data = pd.read_csv('export_dataframe.csv')
clean_data.head()

,Ad_Type,Regions,Advertiser_Name,Date_Range_Start,Date_Range_End,Num_of_Days,Impressions,Spend_USD,Age_Targeting,Gender_Targeting,Geo_Targeting_Included,Geo_Targeting_Excluded,Spend_Range_Min_USD,Spend_Range_Max_USD
0,Text,US,ALEX FOR AZ,2020-07-08,2020-08-04,28,≤ 10k,1k-50k,Not targeted,Not targeted,"85054, 85207, 85215, 85250, 85251, 85253, 8525...",Not targeted,1000,50000.0
1,Text,US,ALEX FOR AZ,2020-07-07,2020-08-04,29,≤ 10k,1k-50k,Not targeted,Not targeted,"85054, 85207, 85215, 85250, 85251, 85253, 8525...",Not targeted,1000,50000.0
2,Text,US,CITIZENS FOR JD WOOTEN,2020-08-28,2020-11-03,64,10k-100k,1k-50k,Not targeted,Not targeted,"27215, 27217, 27233, 27244, 27249, 27253, 2725...",Not targeted,1000,50000.0
3,Video,US,CITIZENS FOR JD WOOTEN,2020-10-28,2020-11-03,7,100k-1M,1k-50k,"18-24, 25-34, 35-44, 45-54, 55-64, ≥65, Unknow...","Male, Female, Unknown gender","27215, 27217, 27233, 27244, 27249, 27253, 2725...",Not targeted,1000,50000.0
4,Video,US,CITIZENS FOR JD WOOTEN,2020-10-26,2020-11-03,9,10k-100k,1k-50k,"18-24, 25-34, 35-44, 45-54, 55-64, ≥65, Unknow...","Male, Female, Unknown gender","27215, 27217, 27233, 27244, 27249, 27253, 2725...",Not targeted,1000,50000.0


In [10]:
zips =  pd.read_csv('free-zipcode-database-Primary.csv')
zips.head()

,Zipcode,ZipCodeType,City,State,LocationType,Lat,Long,Location,Decommisioned,TaxReturnsFiled,EstimatedPopulation,TotalWages
0,705,STANDARD,AIBONITO,PR,PRIMARY,18.14,-66.26,NA-US-PR-AIBONITO,False,NaN,NaN,NaN
1,610,STANDARD,ANASCO,PR,PRIMARY,18.28,-67.14,NA-US-PR-ANASCO,False,NaN,NaN,NaN
2,611,PO BOX,ANGELES,PR,PRIMARY,18.28,-66.79,NA-US-PR-ANGELES,False,NaN,NaN,NaN
3,612,STANDARD,ARECIBO,PR,PRIMARY,18.45,-66.73,NA-US-PR-ARECIBO,False,NaN,NaN,NaN
4,601,STANDARD,ADJUNTAS,PR,PRIMARY,18.16,-66.72,NA-US-PR-ADJUNTAS,False,NaN,NaN,NaN


In [11]:
zips['State'].unique()

array(['PR', 'NJ', 'NY', 'AE', 'VI', 'MA', 'ME', 'NH', 'VT', 'CT', 'RI',
       'DE', 'PA', 'WV', 'KY', 'TN', 'VA', 'GA', 'IN', 'OH', 'IL', 'IA',
       'MN', 'WI', 'MT', 'ND', 'SD', 'KS', 'MO', 'NE', 'CO', 'WY', 'ID',
       'UT', 'AZ', 'NM', 'TX', 'CA', 'NV', 'OR', 'WA', 'AK', 'GU', 'HI',
       'AS', 'PW', 'FM', 'MP', 'MH', 'FL', 'SC', 'AL', 'MS', 'LA', 'AR',
       'OK', 'MI', 'DC', 'MD', 'NC', 'AA', 'AP'], dtype=object)

In [12]:
type(clean_data.loc[0, 'Geo_Targeting_Included'])

str

In [36]:
list_zips = [item.split(', ') for item in clean_data['Geo_Targeting_Included']]

In [24]:
#trying to change zipcode lists into cities; takes a ton of time

zip_column = []
for x in range(len(list_zips)):
    list_of_zips = []
    for zip in list_zips[x]:
        if zip.isdigit():
            int_zip = int(zip)
            zips_item = zips.loc[zips['Zipcode'] == int_zip]
            if(len(zips_item) == 1):
                list_of_zips.append(zips_item['City'].item())
        else:
            list_of_zips.append(zip)
    zip_column.append(set(list_of_zips))

KeyboardInterrupt: 